In [1]:
import numpy as np
import pickle
import tensorflow as tf

from util import SimpleArrayIterator
from datetime import datetime
print('Imports done.')

Imports done.


In [2]:
print('Loading pickles...')
train_question_1 = pickle.load(open('train_question_1.pkl', 'r'))
train_question_2 = pickle.load(open('train_question_2.pkl', 'r'))
train_labels = pickle.load(open('train_labels.pkl', 'r'))
map_index_vec = pickle.load(open('map_index_vec.pkl', 'r'))
print('Done.')

Loading pickles...
Done.


In [3]:
checkpoint_path = './checkpoints/'
logdir = './logs_2'
learning_rate = 0.001
batch_size = 128
dim = 50
N = len(train_question_1)

In [4]:
n_symbols = len(map_index_vec)
embedding_weights = np.zeros((n_symbols, 50), dtype=np.float32)
for index, vec in map_index_vec.items():
    embedding_weights[index,:] = vec

In [ ]:
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

with tf.name_scope("data"):
    q1 = tf.placeholder(tf.float32, shape=[batch_size, None], name='q1')
    q2 = tf.placeholder(tf.float32, shape=[batch_size, None], name='q2')
    target = tf.placeholder(tf.float32, shape=[batch_size], name='target')
    print target

with tf.variable_scope('embedding') as scope:
    embed_q1 = tf.nn.embedding_lookup(embedding_weights, q1)
    scope.reuse_variables()
    embed_q2 = tf.nn.embedding_lookup(embedding_weights, q2)

with tf.variable_scope('lstm') as scope:
    reducemax_q1 = tf.reduce_max(conv1_q1, axis=[1,2])

with tf.variable_scope('reducemax_q2') as scope:
    reducemax_q2 = tf.reduce_max(conv1_q2, axis=[1,2])
    print reducemax_q2

with tf.variable_scope('lr') as scope:
    w = tf.Variable(tf.truncated_normal([2*CONV_OPS,1], stddev=0.1), name='weights')
    b = tf.Variable(tf.zeros([1]), name="bias")

    x = tf.concat(values=[reducemax_q1,reducemax_q2],axis=1)
    logits = tf.matmul(x, w) + b
    logits = tf.reshape(logits, [-1])

with tf.name_scope('lr') as scope:
    cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=target))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy,global_step=global_step)

correct_prediction = tf.equal(tf.round(tf.sigmoid(logits)), target)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", cross_entropy)
    tf.summary.scalar("accuracy", accuracy)
    tf.summary.histogram("histogram_loss", cross_entropy)
    summary_op = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    saver = tf.train.Saver()
    it = PaddedDataIterator(inputs)

    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)

    ckpt = tf.train.get_checkpoint_state(os.path.dirname(checkpoint_path))
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        print global_step.eval()

    writer = tf.summary.FileWriter('./my_graph', sess.graph)

    for i in range(iterations):
        batch = it.next_batch(batch_size)

        if i % 10 == 0:
            print batch['vec1'].shape
            train_accuracy = sess.run(accuracy, feed_dict={q1: batch['vec1'],
                target: batch['is_duplicate'],
                q2: batch['vec2']})
            print('Step %d: Training accuracy %g' % (i, train_accuracy))
            print("{} Saving checkpoint of model...".format(datetime.now()))

            #save checkpoint of the model
            checkpoint_name = os.path.join(checkpoint_path, 'model_step'+str(i)+'.ckpt')
            save_path = saver.save(sess, checkpoint_name)

            print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))

        _,summary = sess.run([train_step,summary_op], feed_dict={q1: batch['vec1'], q2: batch['vec2'], target: batch['is_duplicate']})
        writer.add_summary(summary, global_step = global_step.eval())

    checkpoint_name = os.path.join(checkpoint_path, 'model_train.ckpt')
    save_path = saver.save(sess, checkpoint_name)
writer.close()